In [1]:
# Charger le fichier CSV (modifiez 'path_to_file.csv' avec votre propre chemin de fichier)
file_path = 'dataMunich/ACCData/device3-2024-09-14T18_29_05.csv'

In [2]:
import os
import json
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Étape 1: Fonction pour extraire les caractéristiques des données d'entraînement (uniquement l'accéléromètre)
def extract_features_train(data):
    accel = np.array([entry['accel'] for entry in data])
    
    # Extraction des caractéristiques (moyenne, écart-type, min, max pour l'accéléromètre)
    features = []
    features.append(np.mean(accel, axis=0))
    features.append(np.std(accel, axis=0))
    features.append(np.min(accel, axis=0))
    features.append(np.max(accel, axis=0))
    
    return np.concatenate(features)

# Étape 2: Charger les données d'entraînement
def load_data(folder):
    X = []
    y = []
    
    for filename in os.listdir(folder):
        if filename.endswith('.json'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                json_data = json.load(f)
                features = extract_features_train(json_data['data'])  # Utilise uniquement l'accélération
                X.append(features)
                y.append(1 if json_data['iscrash'] else 0)
    
    return np.array(X), np.array(y)

# Étape 3: Entraînement du modèle
folder_path = './dataNous'
X, y = load_data(folder_path)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normaliser/standardiser les caractéristiques
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Entraîner un classificateur (Random Forest dans ce cas)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Prédire les probabilités (pour appliquer des seuils ensuite)
y_prob = clf.predict_proba(X_test)[:, 1]  # Probabilité pour la classe 'crash'

# Fonction pour prédire en fonction d'un seuil
def predic_with_threshold(probs, threshold):
    return np.array(probs > threshold, dtype=int)

# Sélection du meilleur seuil (exemple: 0.7 trouvé plus tôt)
threshold = 0.7

# Étape 4: Fonction pour extraire les caractéristiques d'un fichier de test CSV
def extract_features_from_test(df):
    # Utiliser les colonnes X, Y, Z correspondant aux valeurs de l'accélération
    accel = df[['X (m/s²)', 'Y (m/s²)', 'Z (m/s²)']].values

    # Extraction des caractéristiques (moyenne, écart-type, min, max)
    features = []
    features.append(np.mean(accel, axis=0))
    features.append(np.std(accel, axis=0))
    features.append(np.min(accel, axis=0))
    features.append(np.max(accel, axis=0))
    
    return np.concatenate(features)

# Étape 5: Pipeline complète pour charger un fichier de test et effectuer une prédiction
# file_path = input("Entrez le nom du fichier CSV à tester (ex: 'dataMunich/ACCData/device1-2024-08-19T07_34_34.csv') : ")
df_test = pd.read_csv(file_path)

# Retirer la colonne "received_at"
df_test = df_test.drop(columns=['received_at','cloud_event_id'])

# Simplifier les noms des colonnes commençant par "MESSAGES"
df_test.columns = [
    col.split('.')[-1].replace('ACC_', '') if 'MESSAGES' in col else col
    for col in df_test.columns
]
# Afficher les nouvelles colonnes et quelques lignes pour vérifier
# print(df_test.head())

# Extraire les caractéristiques des données de test
X_test_new = extract_features_from_test(df_test)

# Normaliser les nouvelles données de test
X_test_new_scaled = scaler.transform([X_test_new])

# Prédire les probabilités pour la classe 'iscrash' (1)
y_prob_new = clf.predict_proba(X_test_new_scaled)[:, 1]

# Appliquer le seuil pour la prédiction
y_pred_new = predic_with_threshold(y_prob_new, threshold)

# Afficher la prédiction finale
if y_pred_new[0] == 1:
    print(f"Le fichier {file_path} contient dees données montrant : \nPrédiction: Crash détecté")
else:
    print(f"Le fichier {file_path} contient dees données montrant : \nPrédiction: Pas de crash")




Le fichier dataMunich/ACCData/device3-2024-09-14T18_29_05.csv contient dees données montrant : 
Prédiction: Crash détecté


In [3]:
# Optionnel: Si tu veux vérifier sur une donnée d'entraînement
print("\n--- Test sur une donnée d'entraînement pour vérification ---")
random_index = random.randint(0, len(X_train) - 1)
X_train_sample = X_train[random_index].reshape(1, -1)
y_train_actual = y_train[random_index]

y_prob_train_sample = clf.predict_proba(X_train_sample)[:, 1]
y_pred_train_sample = predic_with_threshold(y_prob_train_sample, threshold)

print(f"Vraie étiquette: {'Crash' if y_train_actual == 1 else 'Pas de crash'}")
if y_pred_train_sample[0] == 1:
    print("Prédiction: Crash détecté")
else:
    print("Prédiction: Pas de crash")


--- Test sur une donnée d'entraînement pour vérification ---
Vraie étiquette: Crash
Prédiction: Crash détecté
